In [1]:
import tensorflow as tf
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

# Vanishing/Exploding Gradients Problem    
## Xavier and He Initialization  
  ![](handson-ml3-main/images/deep/Screenshot%202022-10-26%20142237.png)  

For the uniform distribution, just use r = $√3σ^{2}$. 
   
![](handson-ml3-main/images/deep/Screenshot%202022-10-26%20142610.png)

In [2]:
dense = tf.keras.layers.Dense(50, activation='relu', kernel_initializer='he_normal')

you can obtain any of the initializations listed in Table 11-1
and more using the `VarianceScaling` initializer

In [4]:
he_avg_init = tf.keras.initializers.variance_scaling(scale=2., mode='fan_avg', distribution='uniform')
dense = tf.keras.layers.Dense(50, activation='relu', kernel_initializer=he_avg_init)

## Nonsaturating Activation Functions  
### Leaky ReLU

In [5]:
leaky_relu = tf.keras.layers.LeakyReLU(alpha=.2)    # defaults to alpha=0.3
dense = tf.keras.layers.Dense(50, activation=leaky_relu, kernel_initializer='he_normal')

# model = tf.keras.models.Sequential([
#     # [...]  # more layers
#     tf.keras.layers.Dense(50, kernel_initializer="he_normal"),  # no activation
#     tf.keras.layers.LeakyReLU(alpha=0.2),  # activation as a separate layer
#     # [...]  # more layers
# ])

### ELU
*Outperforms all RELU variants ***but*** Computationaly Expensive*  
Implementing `ELU` in TensorFlow is trivial, just specify the activation function when building each layer, and use `He initialization`:

In [6]:
dense = tf.keras.layers.Dense(50, activation='elu', kernel_initializer='he_normal')

### SELU
Too many constraints too outperform `ELU`

In [7]:
dense = tf.keras.layers.Dense(50, activation='selu', kernel_initializer='lecun_normal')  # lecun_normal

### GELU, Swish and Mish

# Batch Normalization

In [11]:
# extra code - clear the name counters and set the random seed
tf.keras.backend.clear_session()
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)              4

In [12]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [13]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

# extra code – just show that the model works! 😊
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
              metrics="accuracy")
model.fit(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 17s 8ms/step - loss: 0.5507 - accuracy: 0.8091 - val_loss: 0.3932 - val_accuracy: 0.8570
Epoch 2/2
1719/1719 [==============================] - 15s 9ms/step - loss: 0.4072 - accuracy: 0.8563 - val_loss: 0.3597 - val_accuracy: 0.8700


## Gradient Clipping
All `tf.keras.optimizers` accept `clipnorm` or `clipvalue` arguments:

In [14]:
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)

optimizer = tf.keras.optimizers.SGD(clipnorm=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)

## Reusing Pretrained Layers
### Reusing a Keras model

In [15]:
# extra code – split Fashion MNIST into tasks A and B, then train and save
#              model A to "my_model_A".

pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A")

Epoch 1/20
1376/1376 [==============================] - 4s 3ms/step - loss: 1.0997 - accuracy: 0.6639 - val_loss: 0.6670 - val_accuracy: 0.7919
Epoch 2/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.5866 - accuracy: 0.8134 - val_loss: 0.4986 - val_accuracy: 0.8350
Epoch 3/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.4768 - accuracy: 0.8483 - val_loss: 0.4297 - val_accuracy: 0.8564
Epoch 4/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.4211 - accuracy: 0.8630 - val_loss: 0.3905 - val_accuracy: 0.8671
Epoch 5/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.3868 - accuracy: 0.8708 - val_loss: 0.3654 - val_accuracy: 0.8732
Epoch 6/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.3633 - accuracy: 0.8779 - val_loss: 0.3484 - val_accuracy: 0.8782
Epoch 7/20
1376/1376 [==============================] - 3s 2ms/step - loss: 0.3464 - accuracy: 0.8818 - val_loss: 0.3331 - val_accuracy:

In [16]:
# extra code – train and evaluate model B, without reusing model A

tf.random.set_seed(42)
model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))
model_B.evaluate(X_test_B, y_test_B)

Epoch 1/20
7/7 [==============================] - 1s 53ms/step - loss: 0.9635 - accuracy: 0.4350 - val_loss: 0.8553 - val_accuracy: 0.4609
Epoch 2/20
7/7 [==============================] - 0s 19ms/step - loss: 0.8688 - accuracy: 0.4100 - val_loss: 0.7952 - val_accuracy: 0.4164
Epoch 3/20
7/7 [==============================] - 0s 19ms/step - loss: 0.8034 - accuracy: 0.3800 - val_loss: 0.7502 - val_accuracy: 0.3591
Epoch 4/20
7/7 [==============================] - 0s 17ms/step - loss: 0.7522 - accuracy: 0.3450 - val_loss: 0.7223 - val_accuracy: 0.3966
Epoch 5/20
7/7 [==============================] - 0s 16ms/step - loss: 0.7204 - accuracy: 0.4450 - val_loss: 0.6994 - val_accuracy: 0.5064
Epoch 6/20
7/7 [==============================] - 0s 18ms/step - loss: 0.6934 - accuracy: 0.5300 - val_loss: 0.6797 - val_accuracy: 0.5865
Epoch 7/20
7/7 [==============================] - 0s 18ms/step - loss: 0.6700 - accuracy: 0.5700 - val_loss: 0.6620 - val_accuracy: 0.6469
Epoch 8/20
7/7 [===========

[0.509844958782196, 0.890500009059906]

In [17]:
model_A = tf.keras.models.load_model("my_model_A")
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

Note that `model_B_on_A` and `model_A` actually share layers now, so when we train one, it will update both models. If we want to avoid that, we need to build `model_B_on_A` on top of a *clone* of `model_A`:

In [18]:
tf.random.set_seed(42)  # extra code – ensure reproducibility

model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

# extra code – creating model_B_on_A just like in the previous cell
model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])

In [19]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 53ms/step - loss: 1.1162 - accuracy: 0.2600 - val_loss: 1.0024 - val_accuracy: 0.2700
Epoch 2/4
7/7 [==============================] - 0s 20ms/step - loss: 1.0503 - accuracy: 0.1550 - val_loss: 0.9671 - val_accuracy: 0.2117
Epoch 3/4
7/7 [==============================] - 0s 18ms/step - loss: 1.0131 - accuracy: 0.1500 - val_loss: 0.9496 - val_accuracy: 0.1800
Epoch 4/4
7/7 [==============================] - 0s 18ms/step - loss: 0.9760 - accuracy: 0.1500 - val_loss: 0.9262 - val_accuracy: 0.1978
Epoch 1/16
7/7 [==============================] - 1s 40ms/step - loss: 0.9138 - accuracy: 0.1650 - val_loss: 0.8080 - val_accuracy: 0.3650
Epoch 2/16
7/7 [==============================] - 0s 16ms/step - loss: 0.8070 - accuracy: 0.3100 - val_loss: 0.7187 - val_accuracy: 0.4639
Epoch 3/16
7/7 [==============================] - 0s 16ms/step - loss: 0.7190 - accuracy: 0.4750 - val_loss: 0.6672 - val_accuracy: 0.6192
Epoch 4/16
7/7 [===============

In [20]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.3392 - accuracy: 0.9330


[0.3391644358634949, 0.9330000281333923]

# Faster Optimizers

In [21]:
def build_model(seed=42):
    tf.random.set_seed(seed)
    return tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28,28]),
    tf.keras.layers.Dense(100, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(100, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(100, activation='swish', kernel_initializer='he_normal'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

def build_and_train_model(optimizer):
    model = build_model()
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [22]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

history_sgd = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7735 - accuracy: 0.7391 - val_loss: 0.5377 - val_accuracy: 0.8086
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5007 - accuracy: 0.8263 - val_loss: 0.4708 - val_accuracy: 0.8280
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4520 - accuracy: 0.8414 - val_loss: 0.4376 - val_accuracy: 0.8450
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4229 - accuracy: 0.8516 - val_loss: 0.3975 - val_accuracy: 0.8594
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3999 - accuracy: 0.8589 - val_loss: 0.3887 - val_accuracy: 0.8588
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3859 - accuracy: 0.8617 - val_loss: 0.3892 - val_accuracy: 0.8632
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3691 - accuracy: 0.8664 - val_loss: 0.3888 - val_accuracy:

## Momentum optimization

In [23]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

history_momentum = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7667 - accuracy: 0.7359 - val_loss: 0.5382 - val_accuracy: 0.7954
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4976 - accuracy: 0.8276 - val_loss: 0.4664 - val_accuracy: 0.8260
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4500 - accuracy: 0.8418 - val_loss: 0.4381 - val_accuracy: 0.8444
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4225 - accuracy: 0.8520 - val_loss: 0.4017 - val_accuracy: 0.8544
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3996 - accuracy: 0.8578 - val_loss: 0.3969 - val_accuracy: 0.8560
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3855 - accuracy: 0.8631 - val_loss: 0.4006 - val_accuracy: 0.8560
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3693 - accuracy: 0.8673 - val_loss: 0.3804 - val_accuracy:

## Nesterov Accelerated Gradient

In [24]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9,
                                    nesterov=True)

history_momentum = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7538 - accuracy: 0.7431 - val_loss: 0.5243 - val_accuracy: 0.8182
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4871 - accuracy: 0.8303 - val_loss: 0.4610 - val_accuracy: 0.8310
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4436 - accuracy: 0.8446 - val_loss: 0.4278 - val_accuracy: 0.8460
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4181 - accuracy: 0.8531 - val_loss: 0.4015 - val_accuracy: 0.8548
Epoch 5/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3985 - accuracy: 0.8580 - val_loss: 0.3857 - val_accuracy: 0.8576
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3856 - accuracy: 0.8630 - val_loss: 0.4189 - val_accuracy: 0.8524
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3711 - accuracy: 0.8673 - val_loss: 0.4180 - val_accuracy:

## AdaGrad

In [25]:
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)

history_adagrad = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 1.1503 - accuracy: 0.6254 - val_loss: 0.7478 - val_accuracy: 0.7572
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6852 - accuracy: 0.7733 - val_loss: 0.6184 - val_accuracy: 0.7972
Epoch 3/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.5997 - accuracy: 0.7965 - val_loss: 0.5656 - val_accuracy: 0.8092
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5589 - accuracy: 0.8084 - val_loss: 0.5377 - val_accuracy: 0.8144
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5338 - accuracy: 0.8152 - val_loss: 0.5187 - val_accuracy: 0.8166
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5161 - accuracy: 0.8200 - val_loss: 0.5081 - val_accuracy: 0.8224
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5029 - accuracy: 0.8246 - val_loss: 0.4936 - val_accuracy:

## RMSProp

In [26]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

history_rmsprop = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4960 - accuracy: 0.8211 - val_loss: 0.4427 - val_accuracy: 0.8360
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3703 - accuracy: 0.8660 - val_loss: 0.4204 - val_accuracy: 0.8408
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3374 - accuracy: 0.8774 - val_loss: 0.3851 - val_accuracy: 0.8622
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3229 - accuracy: 0.8868 - val_loss: 0.4020 - val_accuracy: 0.8696
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3096 - accuracy: 0.8897 - val_loss: 0.3525 - val_accuracy: 0.8752
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3008 - accuracy: 0.8935 - val_loss: 0.3928 - val_accuracy: 0.8748
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2970 - accuracy: 0.8940 - val_loss: 0.4083 - val_accuracy:

## Adam Optimization

In [27]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9,
                                     beta_2=0.999)

history_adam = build_and_train_model(optimizer)  # extra code                                     

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4876 - accuracy: 0.8244 - val_loss: 0.4176 - val_accuracy: 0.8322
Epoch 2/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3582 - accuracy: 0.8663 - val_loss: 0.3833 - val_accuracy: 0.8486
Epoch 3/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3208 - accuracy: 0.8812 - val_loss: 0.3353 - val_accuracy: 0.8812
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2937 - accuracy: 0.8911 - val_loss: 0.3198 - val_accuracy: 0.8818
Epoch 5/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2744 - accuracy: 0.8962 - val_loss: 0.3117 - val_accuracy: 0.8844
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2594 - accuracy: 0.9029 - val_loss: 0.3201 - val_accuracy: 0.8884
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2444 - accuracy: 0.9068 - val_loss: 0.3296 - val_accuracy:

**Adamax Optimization**

In [28]:
optimizer = tf.keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9,
                                       beta_2=0.999)

history_adamax = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5353 - accuracy: 0.8105 - val_loss: 0.4584 - val_accuracy: 0.8250
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3967 - accuracy: 0.8575 - val_loss: 0.4093 - val_accuracy: 0.8456
Epoch 3/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3602 - accuracy: 0.8682 - val_loss: 0.3595 - val_accuracy: 0.8706
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3341 - accuracy: 0.8778 - val_loss: 0.3395 - val_accuracy: 0.8744
Epoch 5/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3138 - accuracy: 0.8837 - val_loss: 0.3284 - val_accuracy: 0.8802
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2992 - accuracy: 0.8902 - val_loss: 0.3323 - val_accuracy: 0.8804
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.2855 - accuracy: 0.8939 - val_loss: 0.3340 - val_accuracy:

**Nadam Optimization**

In [29]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9,
                                      beta_2=0.999)

history_nadam = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4717 - accuracy: 0.8275 - val_loss: 0.3873 - val_accuracy: 0.8516
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3444 - accuracy: 0.8721 - val_loss: 0.3872 - val_accuracy: 0.8482
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3072 - accuracy: 0.8851 - val_loss: 0.3602 - val_accuracy: 0.8702
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2852 - accuracy: 0.8944 - val_loss: 0.3328 - val_accuracy: 0.8806
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2662 - accuracy: 0.8995 - val_loss: 0.3133 - val_accuracy: 0.8828
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2491 - accuracy: 0.9053 - val_loss: 0.3440 - val_accuracy: 0.8790
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2373 - accuracy: 0.9094 - val_loss: 0.3386 - val_accuracy:

**AdamW Optimization**

In [30]:
import tensorflow_addons as tfa

optimizer = tfa.optimizers.AdamW(weight_decay=1e-5, learning_rate=0.001,
                                 beta_1=0.9, beta_2=0.999)


history_adamw = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4827 - accuracy: 0.8253 - val_loss: 0.4048 - val_accuracy: 0.8372
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3553 - accuracy: 0.8684 - val_loss: 0.3845 - val_accuracy: 0.8538
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3183 - accuracy: 0.8810 - val_loss: 0.3334 - val_accuracy: 0.8768
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2927 - accuracy: 0.8919 - val_loss: 0.3293 - val_accuracy: 0.8788
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2725 - accuracy: 0.8978 - val_loss: 0.3199 - val_accuracy: 0.8808
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2582 - accuracy: 0.9017 - val_loss: 0.3270 - val_accuracy: 0.8828
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2454 - accuracy: 0.9066 - val_loss: 0.3489 - val_accuracy:

## Learning Rate Scheduling
### Power Scheduling
```lr = lr0 / (1 + steps / s)**c```
* Keras uses `c=1` and `s = 1 / decay`

In [33]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01, decay=1e-4, beta_1=0.9,
                                      beta_2=0.999)

history_power_scheduling = build_and_train_model(optimizer)  # extra code

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5524 - accuracy: 0.8025 - val_loss: 0.4922 - val_accuracy: 0.8182
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4474 - accuracy: 0.8433 - val_loss: 0.4820 - val_accuracy: 0.8300
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4190 - accuracy: 0.8533 - val_loss: 0.4455 - val_accuracy: 0.8472
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4116 - accuracy: 0.8592 - val_loss: 0.3992 - val_accuracy: 0.8630
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3968 - accuracy: 0.8633 - val_loss: 0.4219 - val_accuracy: 0.8582
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4095 - accuracy: 0.8628 - val_loss: 0.4838 - val_accuracy: 0.8586
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3974 - accuracy: 0.8648 - val_loss: 0.4767 - val_accuracy:

### Exponential Scheduling
```lr = lr0 * 0.1 ** (epoch / s)```  
* Learning rate will gradually
drop by a factor of 10 every s steps

In [34]:
def exponential_decay_fn(epoch):
    return 0.01 * 0.1 ** (epoch / 20)

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 ** (epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

# extra code – build and compile a model for Fashion MNIST

tf.random.set_seed(42)
model = build_model()
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9,
                                      beta_2=0.999)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train, y_train, epochs=25,
                    validation_data=(X_valid, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/25
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5582 - accuracy: 0.8013 - val_loss: 0.4763 - val_accuracy: 0.8208 - lr: 0.0100
Epoch 2/25
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4354 - accuracy: 0.8478 - val_loss: 0.4465 - val_accuracy: 0.8356 - lr: 0.0089
Epoch 3/25
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3927 - accuracy: 0.8574 - val_loss: 0.4396 - val_accuracy: 0.8482 - lr: 0.0079
Epoch 4/25
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3733 - accuracy: 0.8675 - val_loss: 0.4126 - val_accuracy: 0.8530 - lr: 0.0071
Epoch 5/25
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3568 - accuracy: 0.8725 - val_loss: 0.3929 - val_accuracy: 0.8658 - lr: 0.0063
Epoch 6/25
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3437 - accuracy: 0.8762 - val_loss: 0.4161 - val_accuracy: 0.8538 - lr: 0.0056
Epoch 7/25
1719/1719 [==============================] - 6s

### Piecewise Constant Scheduling

In [ ]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

### Performance Scheduling  
Measure the validation error every N steps (just like for early stopping),
and reduce the learning rate by a factor of . when the error stops
dropping.

In [36]:
# extra code – build and compile the model

lr0 = 0.01
model = build_model()
optimizer = tf.keras.optimizers.SGD(learning_rate=lr0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
history = model.fit(X_train, y_train, epochs=25,
                    validation_data=(X_valid, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/25
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7709 - accuracy: 0.7372 - val_loss: 0.5242 - val_accuracy: 0.8174 - lr: 0.0100
Epoch 2/25
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5045 - accuracy: 0.8249 - val_loss: 0.4786 - val_accuracy: 0.8242 - lr: 0.0100
Epoch 3/25
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4480 - accuracy: 0.8417 - val_loss: 0.4395 - val_accuracy: 0.8416 - lr: 0.0100
Epoch 4/25
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4200 - accuracy: 0.8507 - val_loss: 0.4030 - val_accuracy: 0.8512 - lr: 0.0100
Epoch 5/25
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3993 - accuracy: 0.8572 - val_loss: 0.3964 - val_accuracy: 0.8542 - lr: 0.0100
Epoch 6/25
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3820 - accuracy: 0.8628 - val_loss: 0.4041 - val_accuracy: 0.8568 - lr: 0.0100
Epoch 7/25
1719/1719 [==============================] - 4s

### tf.keras schedulers
Updates the learning rate at each step rather
than at each epoch

In [37]:
import math

batch_size = 32
n_epochs = 25
n_steps = n_epochs * math.ceil(len(X_train) / batch_size)
scheduled_learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01, decay_steps=n_steps, decay_rate=0.1)
optimizer = tf.keras.optimizers.SGD(learning_rate=scheduled_learning_rate)

# extra code – build and train the model
model = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7670 - accuracy: 0.7398 - val_loss: 0.5221 - val_accuracy: 0.8212
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5071 - accuracy: 0.8219 - val_loss: 0.4849 - val_accuracy: 0.8256
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4528 - accuracy: 0.8396 - val_loss: 0.4472 - val_accuracy: 0.8430
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4264 - accuracy: 0.8496 - val_loss: 0.4131 - val_accuracy: 0.8580
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4060 - accuracy: 0.8559 - val_loss: 0.4031 - val_accuracy: 0.8606
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3904 - accuracy: 0.8617 - val_loss: 0.4148 - val_accuracy: 0.8566
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3796 - accuracy: 0.8647 - val_loss: 0.3900 - val_accuracy:

In [38]:
# extra code – shows how to use PiecewiseConstantDecay
# scheduled_learning_rate = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
#     boundaries=[5. * n_steps_per_epoch, 15. * n_steps_per_epoch],
#     values=[0.01, 0.005, 0.001])

# Avoiding Overfitting Through Regularization
## $\ell_1$ and $\ell_2$ regularization

In [ ]:
layer = tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal",
                              kernel_regularizer=tf.keras.regularizers.l2(0.01))

Or use `l1(0.1)` for ℓ<sub>1</sub> regularization with a factor of 0.1, or `l1_l2(0.1, 0.01)` for both ℓ<sub>1</sub> and ℓ<sub>2</sub> regularization, with factors 0.1 and 0.01 respectively.

In [ ]:
# tf.random.set_seed(42)  # extra code – for reproducibility

# from functools import partial

# RegularizedDense = partial(tf.keras.layers.Dense,
#                            activation="relu",
#                            kernel_initializer="he_normal",
#                            kernel_regularizer=tf.keras.regularizers.l2(0.01))

# model = tf.keras.Sequential([
#     tf.keras.layers.Flatten(input_shape=[28, 28]),
#     RegularizedDense(100),
#     RegularizedDense(100),
#     RegularizedDense(10, activation="softmax")
# ])

# # extra code – compile and train the model
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
#               metrics=["accuracy"])
# history = model.fit(X_train, y_train, epochs=2,
#                     validation_data=(X_valid, y_valid))

## Dropout

In [40]:
tf.random.set_seed(42)  # extra code – for reproducibility

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="swish",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="swish",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

# extra code – compile and train the model
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.7058 - accuracy: 0.7484 - val_loss: 0.5415 - val_accuracy: 0.8158
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6395 - accuracy: 0.7785 - val_loss: 0.4800 - val_accuracy: 0.8286
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6201 - accuracy: 0.7861 - val_loss: 0.4804 - val_accuracy: 0.8290
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6133 - accuracy: 0.7854 - val_loss: 0.5043 - val_accuracy: 0.8230
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6184 - accuracy: 0.7868 - val_loss: 0.5461 - val_accuracy: 0.8336
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6329 - accuracy: 0.7812 - val_loss: 0.4977 - val_accuracy: 0.8230
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6374 - accuracy: 0.7831 - val_loss: 0.5090 - val_accuracy:

In [41]:
model.evaluate(X_train, y_train)

1719/1719 [==============================] - 4s 2ms/step - loss: 0.4693 - accuracy: 0.8342


[0.4692531228065491, 0.8342182040214539]

In [42]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.5206 - accuracy: 0.8201


[0.5206251740455627, 0.8201000094413757]

## MC Dropout

In [43]:
tf.random.set_seed(42)  # extra code – for reproducibility

y_probas = np.stack([model(X_test, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)

model.predict(X_test[:1]).round(3)

1/1 [==============================] - 0s 165ms/step


array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.076, 0.   , 0.235, 0.   ,
        0.689]], dtype=float32)

In [44]:
y_proba[0].round(3)

array([0.004, 0.001, 0.001, 0.005, 0.003, 0.111, 0.004, 0.229, 0.003,
       0.638], dtype=float32)

In [45]:
y_std = y_probas.std(axis=0)
y_std[0].round(3)

array([0.022, 0.008, 0.008, 0.031, 0.019, 0.141, 0.021, 0.241, 0.014,
       0.267], dtype=float32)

In [46]:
y_pred = y_proba.argmax(axis=1)
accuracy = (y_pred == y_test).sum() / len(y_test)
accuracy

0.8302

In [47]:
class MCDropout(tf.keras.layers.Dropout):
    def call(self, inputs, training=None):
        return super().call(inputs, training=True)

# extra code – shows how to convert Dropout to MCDropout in a Sequential model
Dropout = tf.keras.layers.Dropout
mc_model = tf.keras.Sequential([
    MCDropout(layer.rate) if isinstance(layer, Dropout) else layer
    for layer in model.layers
])
mc_model.set_weights(model.get_weights())

In [48]:
mc_model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_19 (Flatten)        (None, 784)               0         
                                                                 
 mc_dropout (MCDropout)      (None, 784)               0         
                                                                 
 dense_76 (Dense)            (None, 100)               78500     
                                                                 
 mc_dropout_1 (MCDropout)    (None, 100)               0         
                                                                 
 dense_77 (Dense)            (None, 100)               10100     
                                                                 
 mc_dropout_2 (MCDropout)    (None, 100)               0         
                                                                 
 dense_78 (Dense)            (None, 10)              

In [49]:
# extra code – shows that the model works without retraining
tf.random.set_seed(42)
np.mean([mc_model.predict(X_test[:1])
         for sample in range(100)], axis=0).round(2)

1/1 [==============================] - 0s 25ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.15, 0.  , 0.25, 0.  , 0.6 ]],
      dtype=float32)